In [1]:
import os
import sys

import vector_tile_pb2

/Users/satoru/github.com/nokonoko1203/0002_study/mvt-encode-notebook/.venv/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.3 is exactly one major version older than the runtime version 6.30.2 at vector_tile.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


In [2]:
# --- ZigZag エンコーディング ---
def zigzag_encode(n):
    """整数をZigZagエンコードする"""
    return (n << 1) ^ (n >> 31)  # 32ビット想定。より大きな整数を扱う場合は調整が必要


# --- ジオメトリコマンドとパラメータのエンコードヘルパー ---
def encode_command_integer(command_id, count):
    """コマンドIDとパラメータ数をエンコード"""
    return (command_id & 0x7) | (count << 3)


def encode_parameter_integer(value):
    """座標値をZigZagエンコード"""
    return zigzag_encode(value)

In [3]:
tile = vector_tile_pb2.Tile()
layer = tile.layers.add()

In [4]:
# --- レイヤー情報 ---
layer.version = 2
layer.name = "sample_layer"
layer.extent = 4096

In [5]:
# --- フィーチャ (矩形ポリゴン) ---
feature = layer.features.add()
feature.id = 1  # フィーチャID (任意)
feature.type = vector_tile_pb2.Tile.POLYGON  # ジオメトリタイプ

In [6]:
# --- ジオメトリコマンドの作成 ---
geometry = []
cursor_x, cursor_y = 0, 0  # 現在のカーソル位置 (差分計算用)

In [7]:
# 1. MoveTo(100, 100)
cmd_move_to = encode_command_integer(1, 1)  # Command ID = 1 (MoveTo), Count = 1
param_x = encode_parameter_integer(100)
param_y = encode_parameter_integer(100)
geometry.extend([cmd_move_to, param_x, param_y])
cursor_x, cursor_y = 100, 100

# 2. LineTo(200, 100), LineTo(200, 200), LineTo(100, 200) をまとめて指定
# LineTo は複数点をまとめて指定できる
cmd_line_to = encode_command_integer(2, 3)  # Command ID = 2 (LineTo), Count = 3
geometry.append(cmd_line_to)

# LineTo(200, 100) の相対座標 (+100, 0)
dx1 = 200 - cursor_x
dy1 = 100 - cursor_y
geometry.extend([encode_parameter_integer(dx1), encode_parameter_integer(dy1)])
cursor_x, cursor_y = 200, 100

# LineTo(200, 200) の相対座標 (0, +100)
dx2 = 200 - cursor_x
dy2 = 200 - cursor_y
geometry.extend([encode_parameter_integer(dx2), encode_parameter_integer(dy2)])
cursor_x, cursor_y = 200, 200

# LineTo(100, 200) の相対座標 (-100, 0)
dx3 = 100 - cursor_x
dy3 = 200 - cursor_y
geometry.extend([encode_parameter_integer(dx3), encode_parameter_integer(dy3)])
cursor_x, cursor_y = 100, 200

# 3. ClosePath
cmd_close_path = encode_command_integer(7, 1)  # Command ID = 7 (ClosePath), Count = 1
geometry.append(cmd_close_path)

In [8]:
# エンコードされたジオメトリをフィーチャに追加
feature.geometry.extend(geometry)

In [9]:
# --- プロパティ (属性情報) の作成 ---
# レイヤーのキーリストと値リストを作成
keys = ["name", "type", "area"]
values = [
    vector_tile_pb2.Tile.Value(string_value="MyRectangle"),
    vector_tile_pb2.Tile.Value(string_value="debug"),
    vector_tile_pb2.Tile.Value(double_value=10000.0),  # 矩形の面積 (例)
]

In [10]:
# キーと値をレイヤーに追加
layer.keys.extend(keys)
layer.values.extend(values)

In [11]:
# フィーチャのタグ (キーリストと値リストへのインデックス) を作成
# tags = [key_index_1, value_index_1, key_index_2, value_index_2, ...]
# "name" はインデックス 0, "MyRectangle" はインデックス 0
# "type" はインデックス 1, "debug" はインデックス 1
# "area" はインデックス 2, 10000.0 はインデックス 2
feature.tags.extend([0, 0, 1, 1, 2, 2])

# --- シリアライズしてファイルに書き込み ---
mvt_data = tile.SerializeToString()

In [12]:
output_filename = "./14/8907/5509.mvt"
try:
    os.makedirs(os.path.dirname(output_filename), exist_ok=True)

    with open(output_filename, "wb") as f:
        f.write(mvt_data)
    print(f"MVTファイル '{output_filename}' を作成しました。")
except IOError as e:
    print(f"ファイル書き込みエラー: {e}", file=sys.stderr)


MVTファイル './14/8907/5509.mvt' を作成しました。
